In [1]:
import os

from src.cnnClassifier.config.configuration import ConfigurationManager

In [2]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification'

In [5]:
from dataclasses import  dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [10]:
class ConfigManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"data")
        create_directories([
        Path(training.root_dir)
        ])
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config



In [11]:
import  os
import  urllib as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:

class Training:
    def __init__(self,config:TrainingConfig):
        self.steps_per_epoch = None
        self.train_generator = None
        self.valid_generator = None
        self.model = None
        self.validation_steps = None
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)



    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [13]:
try:
    config = ConfigManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2026-02-17 07:08:25,959]: INFO: common: Yaml file: config\config.yml loaded successfully
[2026-02-17 07:08:25,962]: INFO: common: Yaml file: params.yml loaded successfully
[2026-02-17 07:08:25,963]: INFO: common: Created directory at: artifacts
[2026-02-17 07:08:25,964]: INFO: common: Created directory at: artifacts\training
Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
368/368 [==============================] - 408s 1s/step - loss: 190.7909 - accuracy: 0.7940 - val_loss: 124.0252 - val_accuracy: 0.8510
